Реализация модуля по расчету производительности скважин различного заканчивания на псевдоустановившемся режиме.

Задача со звездочкой к курсу "Методы математической физики в нефтяном инжиниринге".

Выполнил Свойкин Е.В.

In [19]:
import math as m

In [20]:
def q_well(k: float, h: float, Pres: float, Pwf: float, mu: float, B: float, re: float, rw: float, S: float):
    """
    Функция для расчета производительности скважины на псевдоустановившемся режиме

    Parameters
    ----------
    :param k: проницаемость пласта, мД
    :param h: толщина пласта, м
    :param Pres: среднее пластовое давление, атм
    :param Pwf: забойное давление, атм
    :param mu: вякость нефти, сПз
    :param B: объемный коэффициент, м3/м3
    :param re: радиус контура, м
    :param rw: радиус скважины, м
    :param S: скин фактора (расчетный)

    :return q: производительность скважины, м3/сут

    ----------
    """
    return k*h*(Pres-Pwf)/(18.4*mu*B*(m.log1p(re/rw)+S-0.75))

Рассчитаем производительность скважин различного заканчивания на псевдоустановивщемся режиме

In [21]:
# Исходные данные
k = 10 # мД
h = 10 # м
Pres = 250 # атм
Pwf = 50 # атм
mu = 1.01 # сПз
B = 1.228 # м3/м3
re = 500 # м
rw = 0.108 # м

# Параметры для расчета скин-фактора
kd = k/2 # измененная проницаемость, мД
rd = rw*3 # радиус зоны с проницаемостью, измененной по сравнению с проницаемостью пласта, м
model = 0 # Корреляция Papatzacos
hw = 3*h/4 # мощность вскрытого интервала, открытого для притока (0<=hw<=h), м
zw = h/2 # расстояние от подошвы пласта до центра интервала, открытого для притока (hw/2<=zw<=h-hw/2), м
kh = k # проницаемость пласта в латеральном направлении, мД
kv = k/10 # проницаемость пласта в вертикальном направлении, мД
y = 1 # коэффициент несовершенства степени вскрытия

Необсаженная вертикальная скважина

In [22]:
from uncased_vertical_well import UncasedVW
uncased_vertical_well = UncasedVW()
# Совершенная скважина по степени вскрытия
q_perf_uncased_vertical_well = q_well(k, h, Pres, Pwf, mu, B, re, rw, uncased_vertical_well.perfect_s(k, kd, rw, rd))
print(f'Производительность совершенной по степени вскрытия вертикальной скважины - {q_perf_uncased_vertical_well} м3/сут')
# Несовершенная скважина по степени всерытия
q_unperf_uncased_vertical_well = q_well(k, h, Pres, Pwf, mu, B, re, rw, uncased_vertical_well.unperfect_s(model, h, hw, rw, zw, kh, kv, k, kd, rd, y))
print(f'Производительность несовершенной по степени вскрытия вертикальной скважины - {q_unperf_uncased_vertical_well} м3/сут')

Производительность совершенной по степени вскрытия вертикальной скважины - 96.55225418339903 м3/сут
Производительность несовершенной по степени вскрытия вертикальной скважины - 72.55445671914973 м3/сут


In [23]:
# Параметры для расчета скин-фактора
phi = 0 # фазировка перфорационных зарядов, градусы
Lp = 0.1 # длина перфорационных каналов, м
rp = 0.0005 # радиус перфорационных каналов, м
ns = 10 # плотность перфорационных отверстий, отв/м
kcz = 2*k # проницаемость зоны разрушения породы вокруг перфорационных каналов, мД
rcz = 0.2 # радиус зоны разрушения породы вокруг перфорационных каналов, м

Перфорированная вертикальная скважина

In [24]:
from perforated_vertical_well import PerfVW
perf_vertical_well = PerfVW()
# Полностью перфорированная скважина
q_full_perf_vertical_well = q_well(k, h, Pres, Pwf, mu, B, re, rw, perf_vertical_well.full_perf_s(k, kd, rw, rd, phi, Lp, rp, ns, kh, kv, kcz, rcz))
print(f'Производительность полностью перфорированной вертикальной скважины - {q_full_perf_vertical_well} м3/сут')
# Частично перфорированная скважина
q_part_perf_vertical_well = q_well(k, h, Pres, Pwf, mu, B, re, rw, perf_vertical_well.part_perf_s(k, kd, rw, rd, phi, Lp, rp, ns, kh, kv, kcz, rcz, h, hw, model, zw, 1))
print(f'Производительность частично перфорированной вертикальной скважины - {q_part_perf_vertical_well} м3/сут')


Производительность полностью перфорированной вертикальной скважины - 242.2825532326301 м3/сут
Производительность частично перфорированной вертикальной скважины - 182.59773123297762 м3/сут


In [25]:
# Параметры для расчет скин-фактора
teta = 60 # угол отклонения ствола скважины от вертикали, градусы
Lwpc = 200 # длина наклонно-направленной скважины в пределах продуктивного пласта, открытая для притока флюида, м

Необсаженная наклонно-направленная скважина

In [26]:
from unanchored_directional_well import UnanchDW
unanchored_directional_well = UnanchDW()
# Совершенная скважина по степени вскрытия
q_perf_directional_well = q_well(k, h, Pres, Pwf, mu, B, re, rw, unanchored_directional_well.perfect_s(k, kd, rw, rd, model, teta, kh, kv, h, hw, zw))
print(f'Производительность совершенной по степени вскрытия необсаженной наклонно-направленной скважины - {q_perf_directional_well} м3/сут')
# Несовершенная скважина по степени вскрытия
q_unperf_directional_well = q_well(k, h, Pres, Pwf, mu, B, re, rw, unanchored_directional_well.unperfect_s(k, kd, rw, rd, h, Lwpc, teta, kh, kv, hw, zw))
print(f'Производительность несовершенной по степени вскрытия наклонно-направленной скважины - {q_unperf_directional_well} м3/сут')

Производительность совершенной по степени вскрытия необсаженной наклонно-направленной скважины - 137.57663591705716 м3/сут
Производительность несовершенной по степени вскрытия наклонно-направленной скважины - 199.43209342482893 м3/сут


Перфорированная наклонно-направленная скавжина

In [27]:
from perforated_directional_well import PerfDW
perforated_directional_well = PerfDW()
# Полностью перфорированная скважина
q_perf_directional_well = q_well(k, h, Pres, Pwf, mu, B, re, rw, perforated_directional_well.full_perf_s(k, kd, rw, rd, teta, phi, Lp, rp, ns, kh, kv, kcz, rcz, model, h, hw, zw))
print(f'Производительность полностью перфорированной наклонно-направленной скважины - {q_perf_directional_well} м3/сут')
# Частично перфорированная скважина
q_perf_directional_well = q_well(k, h, Pres, Pwf, mu, B, re, rw, perforated_directional_well.part_perf_s(k, kd, rw, rd, teta, phi, Lp, rp, ns, kh, kv, kcz, rcz, h, hw, zw))
print(f'Производительность частично перфорированной наклонно-направленной скважины - {q_perf_directional_well} м3/сут')

Производительность совершенной по степени вскрытия необсаженной наклонно-направленной скважины - 75.74668362872804 м3/сут
Производительность совершенной по степени вскрытия необсаженной наклонно-направленной скважины - 106.81704819641227 м3/сут
